In [1]:
import requests
import xml.etree.ElementTree as ET  # for parsing XML

In [12]:
base_semester_url = "https://courses.illinois.edu/cisapp/explorer/schedule/2023/fall.xml"

subjects_list = []

subjects_root = ET.fromstring(requests.get(base_semester_url).content)
subjects_xml = subjects_root.find("subjects").findall("subject")

In [14]:
for subject in subjects_xml:
    subjects_list.append({
        "name" : subject.text,
        "code" : subject.attrib["id"],
        "link" : subject.attrib["href"]
    })

In [16]:
courses_list = []

for subject in subjects_list : 
    course_root = ET.fromstring(requests.get(subject["link"]).content)
    course_xml = course_root.find("courses").findall("course")
    for course in course_xml:
        courses_list.append({
            "name" : course.text,
            "number" : course.attrib["id"],
            "code" : subject["code"],
            "link" : course.attrib["href"]
        })


In [17]:
print(courses_list)

[{'name': 'Intro Asian American Studies', 'number': '100', 'code': 'AAS', 'link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2023/fall/AAS/100.xml'}, {'name': 'U.S. Race and Empire', 'number': '200', 'code': 'AAS', 'link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2023/fall/AAS/200.xml'}, {'name': 'US Racial & Ethnic Politics', 'number': '201', 'code': 'AAS', 'link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2023/fall/AAS/201.xml'}, {'name': 'US Citizenship Comparatively', 'number': '215', 'code': 'AAS', 'link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2023/fall/AAS/215.xml'}, {'name': 'Intro Asian American Theatre', 'number': '260', 'code': 'AAS', 'link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2023/fall/AAS/260.xml'}, {'name': 'Constructing Race in America', 'number': '281', 'code': 'AAS', 'link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2023/fall/AAS/281.xml'}, {'name': 'Asian American Literature', 'num

In [19]:
import pymongo
client = pymongo.MongoClient(mongodb_uri)
db = client["schedule"]

In [21]:
classes = db["classes"]
for course in courses_list:
    classes.insert_one({
        "name" : course["name"],
        "number" : course["number"],
        "code" : course["code"],
        "api_link" : course["link"],
        "year" : 2023,
        "semester" : "fall"
    })